CS524: Introduction to Optimization Lecture 14
======================================

## Michael Ferris<br> Computer Sciences Department <br> University of Wisconsin-Madison

## October 9, 2023
--------------

In [1]:
%load_ext gams.magic

#  Widgetco

Widgetco is about to introduce a new product.  One unit of this product is produced by assembling subassembly 1 and subassembly 2. Before production begins on either subassembly, raw materials must be purchased and workers must be trained. Before the subassemblies can be assembled into the final product, the finished subassembly 2 must be inspected.  The GAMS code that sets up this data is given below.

In [2]:
%%gams
set activity /
    A "Train Workers",
    B "Purchase Raw Materials",
    C "Make Subassembly 1",
    D "Make Subassembly 2",
    E "Inspect Subassembly 2",
    F "Assemble Subassemblies" /;

alias (activity, I,J);

set pred(I,J) "I preceeds J" /
    (A,B) . C,
    (A,B) . D,
    D . E,
    (C,E) . F /;

parameter duration(I) /
    A   6
    B   9
    C   8
    D   7
    E   10
    F   12 /;

In [3]:
import numpy as np

m = gams.exchange_container
activity = m.addSet('activity',records=[('A',"Train Workers"),('B',"Purchase Raw Materials"),
    ('C',"Make Subassembly 1"),('D',"Make Subassembly 2"),
    ('E',"Inspect Subassembly 2"),('F',"Assemble Subassemblies")])
I = m.addAlias('I',activity)
J = m.addAlias('J',activity)
pred = m.addSet('pred',[I,J],description="I preceeds J", records=[('A','C'), ('B','C'), 
    ('A','D'), ('B','D'), ('D','E'), ('C','F'), ('E','F')])
duration = m.addParameter('duration',[I],records=np.array([6, 9, 8, 7, 10, 12]))

In [4]:
display(activity.records,pred.toList(),duration.records)

,uni,element_text
0,A,Train Workers
1,B,Purchase Raw Materials
2,C,Make Subassembly 1
3,D,Make Subassembly 2
4,E,Inspect Subassembly 2
5,F,Assemble Subassemblies


[('A', 'C'),
 ('B', 'C'),
 ('A', 'D'),
 ('B', 'D'),
 ('D', 'E'),
 ('C', 'F'),
 ('E', 'F')]

,I,value
0,A,6.0
1,B,9.0
2,C,8.0
3,D,7.0
4,E,10.0
5,F,12.0


In [5]:
%%gams
variables projDur;
positive variable t(I) "time activity starts";

equations incidence(I,J), endTime(I);

incidence(I,J)$pred(I,J)..
    t(J) =g= t(I) + duration(I);

endTime(I)..
    projDur =g= t(I) + duration(I);

model cpm /incidence,endTime/;

Only run the following cell to demonstrate a non-unique critical path

In [6]:
# gams.gams('duration('C') = 17;')

In [7]:
%%gams
solve cpm using lp minimizing projDur;

set critical(activity) "critical activities";

critical(activity) = yes$(smax(J$pred(J,activity),incidence.m(J,activity)) ge 1
            or smax(J$pred(activity,J),incidence.m(activity,J)) ge 1);

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),38.0,13,7,LP,CPLEX,0.016


Critical path is identifed by binding constraints (those with positive multipliers)

In [8]:
incidence=m.data['incidence']
critical=m.data['critical']
display(critical.toList(),incidence.pivot(fill_value='',value='marginal'))

['B', 'D', 'E', 'F']

,C,D,F,E
A,0.0,0.0,,
B,-0.0,1.0,,
C,,,0.0,
D,,,,1.0
E,,,1.0,


Now let's set up the dual

In [9]:
%%gams
positive variables
    lamda(I,J) "Dual var for incidence (critical arc)"
    pi(I);

free variable dual_obj;

equations
    projDur_dual_eq "Dual equation for projDur"
    teq(I)  "Equations for primal t vars"
    dual_obj_def;

dual_obj_def..
    dual_obj =E= sum((I,J)$pred(I,J), lamda(I,J) * duration(I))
        + sum(I,duration(I)*pi(I));

projDur_dual_eq..
    sum(I, pi(I)) =E= 1;

teq(I)..
    -pi(I) - sum(J$pred(I,J), lamda(I,J)) + sum(J$pred(J,I), lamda(J,I)) =L= 0;


model dual_cpm /teq, projDur_dual_eq, dual_obj_def/;

solve dual_cpm using lp maximizing dual_obj;

critical(activity) = yes$(smax(J$pred(J,activity),lamda.l(J,activity)) ge 1
            or smax(J$pred(activity,J),lamda.l(activity,J)) ge 1);

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),38.0,8,14,LP,CPLEX,0.001


In [10]:
lamda=m.data['lamda']
critical=m.data['critical']
display(lamda.pivot(fill_value=''),critical.toList())

,C,D,F,E
A,0.0,0.0,,
B,0.0,1.0,,
C,,,0.0,
D,,,,1.0
E,,,1.0,


['B', 'D', 'E', 'F']

Now try to find early and late event times

In [11]:
%%gams
parameter
    eeTime(activity) "early event time",
    leTime(activity) "late event time";

projDur.fx = projDur.l;

variables objective;
equations timeopt;

timeopt..
    objective =e= sum(activity,t(activity));

model eventtimes /timeopt,incidence,endTime/;

solve eventtimes using lp maximizing objective;
leTime(activity) = t.l(activity);
solve eventtimes using lp minimizing objective;
eeTime(activity) = t.l(activity);

critical(activity) = yes$(eeTime(activity) ge leTime(activity));

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),72.0,14,8,LP,CPLEX,0.001
1,Normal (1),Optimal Global (1),60.0,14,8,LP,CPLEX,0.001


In [12]:
eeTime=m.data['eeTime']
leTime=m.data['leTime']
critical=m.data['critical']
display(eeTime.records,leTime.records,critical.toList())

,activity,value
0,C,9.0
1,D,9.0
2,E,16.0
3,F,26.0


,activity,value
0,A,3.0
1,C,18.0
2,D,9.0
3,E,16.0
4,F,26.0


['B', 'D', 'E', 'F']

Now do this only solving the single LP (i.e. only need data and the value projDur.l)
(Note this is more complicated gams programming)

In [13]:
%%gams
set firstdone(activity), next(activity);

eeTime(activity) = -inf;

firstdone(J) = yes$(sum(I$pred(I,J),1) eq 0);
eeTime(firstdone) = 0;

set iters /iter1*iter100/;

loop(iters$(card(firstdone) gt 0),
  next(I) = yes$sum(pred(firstdone,I),1);
  eeTime(next) = smax(pred(J,next), eeTime(J)+duration(J));
  firstdone(activity) = next(activity);
);

set lastdone(activity), prev(activity);

leTime(activity) = inf;

lastdone(I) = yes$(sum(J$pred(I,J),1) eq 0);
leTime(lastdone) = projDur.l-duration(lastdone);

loop(iters$(card(lastdone) gt 0),
  prev(I) = yes$sum(pred(I,lastdone),1);
  leTime(prev) = smin(pred(prev,J), leTime(J)-duration(prev));
  lastdone(activity) = prev(activity);
);

critical(activity) = yes$(eeTime(activity) ge leTime(activity));

In [14]:
eeTime=m.data['eeTime']
leTime=m.data['leTime']
critical=m.data['critical']
display(eeTime.records,leTime.records,critical.toList())

,activity,value
0,C,9.0
1,D,9.0
2,E,16.0
3,F,26.0


,activity,value
0,A,3.0
1,C,18.0
2,D,9.0
3,E,16.0
4,F,26.0


['B', 'D', 'E', 'F']

In [15]:
gams.gams_cleanup('--closedown')